In [1]:
%matplotlib widget
import numpy as np
import os
import matplotlib.pyplot as plt
import scipy.spatial
from sklearn.cluster import DBSCAN
import pclpy
from pclpy import pcl
import NormalEstimatorHough as Estimator
import time

import planeslam.io as io
import planeslam.general as general
from planeslam.clustering import cluster_mesh_graph_search, plot_clusters

%load_ext autoreload
%autoreload 2

In [18]:
binpath = os.path.join(os.getcwd(),'..', 'data', 'airsim', 'blocks_20_samples_1', 'lidar', 'Drone0')
PCs = io.read_lidar_bin(binpath)
P = PCs[0]
#P = general.NED_to_ENU(P)
(n,_) = P.shape

In [19]:
# Plot the points
general.plot_pc(P)

In [20]:
# Estimate normals via Hough
start = time.time()
estimator = Estimator.NormalEstimatorHough()
estimator.set_points(P)
estimator.set_K(50)
estimator.estimate_normals()
normals = estimator.get_normals()
print("Took",time.time()-start,"seconds")

Took 0.05900859832763672 seconds


In [26]:
# Plot normals
general.plot_normals(P,normals)
# plt_normals = 10*estimator.get_normals()
# ax = general.plot_3D_setup(P)
# ax.scatter3D(P[:,0], P[:,1], P[:,2], marker='.')  
# ax.quiver(P[:,0],P[:,1],P[:,2],plt_normals[:,0],plt_normals[:,1],plt_normals[:,2],color='r')

In [ ]:
# Use pclpy region growing to extract planes
pc = pcl.PointCloud.PointXYZ.from_array(P)
rg = pcl.segmentation.RegionGrowing.PointXYZ_Normal()
rg.setInputCloud(pc)
rg.setInputNormals(pcl.PointCloud.Normal.from_array(normals))
rg.setMaxClusterSize(1000000)
rg.setMinClusterSize(10)
rg.setNumberOfNeighbours(20)
rg.setSmoothnessThreshold(2/ 180 * np.pi)
rg.setCurvatureThreshold(3)
rg.setResidualThreshold(1)
start_time = time.time()
clusters = pcl.vectors.PointIndices()
rg.extract(clusters)
print("Elapsed time: ", time.time() - start_time)
print("Number of clusters: ", len(clusters))

In [ ]:
ax = general.plot_3D_setup(P)

for i, c in enumerate(clusters):
    idx = c.indices
    ax.scatter3D(pc.xyz[idx,0], pc.xyz[idx,1], pc.xyz[idx,2], color='C'+str(i), marker='.')
    

In [ ]:
# Find all neighborhoods
neighbor_radius = 5.0
start = time.time()
kdtree = scipy.spatial.KDTree(P)
pairs = kdtree.query_pairs(neighbor_radius,output_type="ndarray")
#pairs.sort(axis=0)
print("Time taken to get neighbors:",time.time()-start)

In [ ]:
# Collect pairs
col = [[] for i in range(n)]
for (i,j) in pairs:
    col[i].append(j)
    col[j].append(i)

In [ ]:
n

In [ ]:
# Estimate normals
num_ransac = 1
normals = np.zeros_like(P)
for i in range(n):
    if not col[i]:
        continue
    pts = P[col[i],:]
    best_nor = None
    best_fit = np.inf
    for _ in range(num_ransac):
        sub_idx = np.random.choice(range(pts.shape[0]),int(np.ceil(pts.shape[0]/2)),replace=False)
        pts_sub = pts[sub_idx,:]
        mn = pts_sub.mean(axis=0)
        cov = np.sum([np.outer(pts_sub[i,:]-mn,pts_sub[i,:]-mn) for i in range(pts_sub.shape[0])],axis=0)
        (_,S,V) = np.linalg.svd(cov)
        if S[-1] < best_fit:
            best_fit = S[-1]
            best_nor = V[-1,:]
    if P[i,:] @ best_nor > 0:
        best_nor *= -1
    normals[i] = best_nor
    

In [ ]:
# Plot normals 
plt_P = general.downsample(P, factor=5, axis=0)
plt_normals = 10*general.downsample(normals, factor=5, axis=0)
ax = general.plot_3D_setup(P)
ax.scatter3D(plt_P[:,0], plt_P[:,1], plt_P[:,2], marker='.')  
ax.quiver(plt_P[:,0],plt_P[:,1],plt_P[:,2],plt_normals[:,0],plt_normals[:,1],plt_normals[:,2],color='r')